In [2]:
import os
import cv2
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from IPython.display import display, Image
import json # <-- ADDED for saving scan times
import ipywidgets as widgets
from ipywidgets import VBox, HBox

In [3]:
# ----------------------------------------------------------------------
# CELL 2: PATHS & CONFIGURATION
# ----------------------------------------------------------------------

BASE_DIR = r"D:\project\Localens_experiment"
DATASET_DIR = os.path.join(BASE_DIR, "dataset")

# 1. Your database file
CSV_PATH = os.path.join(BASE_DIR, "image_data.csv")

# 2. NEW: File to store the dictionary of last scan times per folder
TIMESTAMPS_FILE = os.path.join(BASE_DIR, "last_scan_times.json")

# 3. Define the database columns --- (THIS IS THE MODIFIED LINE) ---
DB_COLUMNS = ['path', 'folder', 'timestamp', 'R', 'G', 'B', 'objects']

print(f"Base Directory: {BASE_DIR}")
print(f"Dataset Directory: {DATASET_DIR}")
print(f"Database File: {CSV_PATH}")
print(f"Timestamps File: {TIMESTAMPS_FILE}")

Base Directory: D:\project\Localens_experiment
Dataset Directory: D:\project\Localens_experiment\dataset
Database File: D:\project\Localens_experiment\image_data.csv
Timestamps File: D:\project\Localens_experiment\last_scan_times.json


In [4]:
def initialize_database():
    """
    Creates the CSV and JSON files with headers if they don't exist.
    """
    if not os.path.exists(CSV_PATH):
        print(f"Creating new database file: {CSV_PATH}")
        # Create an empty DataFrame with just the columns
        df = pd.DataFrame(columns=DB_COLUMNS)
        df.to_csv(CSV_PATH, index=False)
    
    if not os.path.exists(TIMESTAMPS_FILE):
        print(f"Creating new timestamps file: {TIMESTAMPS_FILE}")
        # Create an empty JSON file
        with open(TIMESTAMPS_FILE, 'w') as f:
            json.dump({}, f)

def load_database():
    """Loads the database CSV."""
    try:
        return pd.read_csv(CSV_PATH)
    except pd.errors.EmptyDataError:
        # File is empty, return a blank DataFrame with correct columns
        return pd.DataFrame(columns=DB_COLUMNS)

def save_database(df):
    """Saves the DataFrame back to the database CSV."""
    df.to_csv(CSV_PATH, index=False)

def load_scan_times():
    """Loads the per-folder timestamp dictionary from the JSON file."""
    try:
        with open(TIMESTAMPS_FILE, 'r') as f:
            return json.load(f)
    except json.JSONDecodeError:
        # File is corrupt or empty, return empty dict
        return {}

def save_scan_times(times_dict):
    """Saves the timestamp dictionary back to the JSON file."""
    with open(TIMESTAMPS_FILE, 'w') as f:
        json.dump(times_dict, f, indent=2)

# --- Run the initialization ---
initialize_database()
print("Database and helper functions are ready.")


Database and helper functions are ready.


In [5]:
# ----------------------------------------------------------------------
# CELL 4: COLOR LOGIC & IMAGE PROCESSING
# ----------------------------------------------------------------------

def classify_color(h, s, v):
    # (Your original function, unchanged)
    h = int(h)
    s = int(s)
    v = int(v)
    h_deg = h * 2 

    if v < 40 and s < 60:
        return "black"
    if v > 230 and s < 30:
        return "white"
    if v > 180 and s < 40:
        return "gray"
    if 15 <= h_deg <= 35:
        if v < 120:
            return "brown"   
        elif 120 <= v <= 200:
            return "orange" 
        else:
            return "yellow"  
    if 35 < h_deg <= 85:
        return "green"
    if 85 < h_deg <= 150:
        return "cyan"
    if 150 < h_deg <= 260:
        return "blue"
    if 260 < h_deg <= 320:
        return "purple"
    if 320 < h_deg <= 345:
        return "pink"
    if h_deg < 15 or h_deg > 345:
        return "red"
    return "unknown"

def extract_dominant_color_lab(image, k=5):
    # (Your original function, unchanged)
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    pixels = lab.reshape(-1, 3)

    kmeans = KMeans(n_clusters=k, n_init=10)
    kmeans.fit(pixels)

    unique, counts = np.unique(kmeans.labels_, return_counts=True)
    dominant_cluster = unique[np.argmax(counts)]
    dominant_color = kmeans.cluster_centers_[dominant_cluster]

    lab_color = np.uint8([[dominant_color]])
    bgr_color = cv2.cvtColor(lab_color, cv2.COLOR_LAB2BGR)
    hsv_color = cv2.cvtColor(bgr_color, cv2.COLOR_BGR2HSV)[0][0]

    h, s, v = hsv_color
    label = classify_color(h, s, v)

    return h, s, v, label

def process_image(image_path):
    """
    MODIFIED: Returns dict matching new columns.
    - Removes H, S, V.
    - Adds 'objects' column as None.
    """
    img = cv2.imread(image_path)
    if img is None:
        return None

    img = cv2.resize(img, (224, 224))
    
    # We still need h, s, v to calculate RGB
    h, s, v, label = extract_dominant_color_lab(img)
    rgb = cv2.cvtColor(np.uint8([[[h, s, v]]]), cv2.COLOR_HSV2RGB)[0][0]
    
    file_mod_time = os.path.getmtime(image_path)

    # --- THIS IS THE MODIFIED RETURN VALUE ---
    return {
        "path": os.path.normpath(image_path),
        "folder": os.path.normpath(os.path.dirname(image_path)),
        "timestamp": file_mod_time,
        "R": int(rgb[0]), "G": int(rgb[1]), "B": int(rgb[2]),
        "objects": None # Set to None (will be 'NaN' in CSV)
    }

print("Image processing functions defined.")

Image processing functions defined.


In [6]:
def scan_for_new_images():
    """
    Scans DATASET_DIR, compares against last scan times,
    and appends only new images to the database.
    """
    print("Running automatic scan for new images...")
    
    # 1. Load current state
    df = load_database()
    scan_times = load_scan_times()
    
    # Use a Set for very fast lookup of paths we already have
    paths_in_db = set(df['path'])
    new_records = []
    
    # This will hold the newest timestamp we find in each folder
    new_max_times = scan_times.copy()

    # 2. Walk through all folders and files
    # We use os.walk as it's the most efficient way
    for dirpath, dirnames, filenames in os.walk(DATASET_DIR):
        
        current_folder = os.path.normpath(dirpath)
        
        # Get the last scan time for this folder, default to 0 if folder is new
        last_scan = scan_times.get(current_folder, 0)
        
        current_max_time_in_folder = last_scan

        # Use tqdm for a progress bar on the files in this folder
        for fname in tqdm(filenames, desc=f"Scanning {os.path.basename(current_folder)}"):
            filepath = os.path.normpath(os.path.join(current_folder, fname))

            # --- CHECK 1: Is it already in our DB? ---
            if filepath in paths_in_db:
                continue

            # --- CHECK 2: Is the file new? ---
            try:
                file_time = os.path.getmtime(filepath)
            except FileNotFoundError:
                continue # File might have been deleted as we were scanning

            if file_time > last_scan:
                # This is a new file!
                # print(f"  [NEW] Found: {fname}") # Uncomment for more detail
                
                # Process it
                result = process_image(filepath)
                if result:
                    new_records.append(result)
            
            # Update the newest time we've seen in this folder
            if file_time > current_max_time_in_folder:
                current_max_time_in_folder = file_time

        # 3. Update the max scan time for this folder if it changed
        if current_max_time_in_folder > last_scan:
            new_max_times[current_folder] = current_max_time_in_folder

    # 4. Save all new data
    if new_records:
        print(f"\nFound {len(new_records)} new images. Appending to database...")
        new_df = pd.DataFrame(new_records, columns=DB_COLUMNS)
        
        # Use pd.concat to append new rows
        df_updated = pd.concat([df, new_df], ignore_index=True)
        
        save_database(df_updated)
        save_scan_times(new_max_times)
        print("Database updated.")
    else:
        print("\nScan complete. No new images found.")
        # We still save scan times, in case new empty folders were found
        save_scan_times(new_max_times)

# --- Run the automatic scan ---
scan_for_new_images()



Running automatic scan for new images...


Scanning dataset: 0it [00:00, ?it/s]
Scanning unlabeled: 100%|██████████████████████████████████████████████████████████████████████| 87/87 [00:00<?, ?it/s]


Scan complete. No new images found.


In [7]:
# ----------------------------------------------------------------------
# CELL 6: CORE - MANUAL "DELETED IMAGE" SCAN (UPGRADED V2)
# ----------------------------------------------------------------------
# (This version cleans both files but does NOT print every
# deleted image path, only a summary.)

def scan_for_deleted_images():
    """
    Loads the database, checks if every file path still exists,
    and removes records for deleted files.
    
    UPGRADED: This function also checks for deleted folders in the
    timestamps JSON and cleans it.
    """
    
    # --- 1. CLEAN THE IMAGE CSV (image_data.csv) ---
    print("Running MANUAL scan for deleted images...")
    
    df = load_database()
    if df.empty:
        print("Database is empty. Nothing to clean.")
    else:
        initial_count = len(df)
        print(f"Checking {initial_count} image records...")

        exists_mask = df['path'].apply(os.path.exists)
        df_clean = df[exists_mask]
        df_deleted = df[~exists_mask]

        deleted_count = len(df_deleted)
        if deleted_count > 0:
            # --- MODIFICATION ---
            # We no longer print every single file.
            print(f"Found {deleted_count} deleted images. Removing from CSV.")
            
            save_database(df_clean)
            print(f"Removed {deleted_count} records. {len(df_clean)} records remain in CSV.")
        else:
            print("No deleted images found in CSV.")

    print("\n" + "-"*30 + "\n")

    # --- 2. CLEAN THE FOLDER JSON (last_scan_times.json) ---
    print("Checking for deleted folders in timestamps file...")
    scan_times = load_scan_times()
    
    if not scan_times:
        print("Timestamps file is empty. Nothing to clean.")
        return

    # We will build a new, clean dictionary
    clean_scan_times = {}
    deleted_folder_count = 0
    
    for folder_path, timestamp in scan_times.items():
        if os.path.exists(folder_path):
            # Folder still exists, keep it
            clean_scan_times[folder_path] = timestamp
        else:
            # Folder is deleted, drop it
            #print(f"  - Deleted folder found: {folder_path}")
            deleted_folder_count += 1
            
    if deleted_folder_count > 0:
        save_scan_times(clean_scan_times)
        print(f"Removed {deleted_folder_count} deleted folders from timestamps JSON.")
    else:
        print("No deleted folders found in timestamps JSON.")

    print("\nManual scan complete.")

# --- To run this, uncomment the line below in this cell ---
# scan_for_deleted_images()

# --- Or, just run it immediately this time ---
scan_for_deleted_images()

print("\nManual delete scan function is ready.")
print("To run it again, type 'scan_for_deleted_images()' in a new cell and run it.")

Running MANUAL scan for deleted images...
Checking 321 image records...
No deleted images found in CSV.

------------------------------

Checking for deleted folders in timestamps file...
No deleted folders found in timestamps JSON.

Manual scan complete.

Manual delete scan function is ready.
To run it again, type 'scan_for_deleted_images()' in a new cell and run it.


In [8]:
scan_for_deleted_images()

Running MANUAL scan for deleted images...
Checking 321 image records...
No deleted images found in CSV.

------------------------------

Checking for deleted folders in timestamps file...
No deleted folders found in timestamps JSON.

Manual scan complete.


In [9]:
# ----------------------------------------------------------------------
# CELL 7: NEW - INTERACTIVE SEARCH WIDGETS
# ----------------------------------------------------------------------
# (This REPLACES your old text-based search cell)

# --- 1. Load the database ---
print("Loading database for search...")
df = load_database()
if df.empty:
    print("WARNING: Database is empty. Please run Cell 5 to scan for images."
         )

# --- 2. Helper function to convert color format ---
def hex_to_rgb(hex_code):
    """Converts a hex color string (e.g., #FF0000) to an (R, G, B) tuple."""
    hex_code = hex_code.lstrip('#')
    return tuple(int(hex_code[i:i+2], 16) for i in (0, 2, 4))

# --- 3. Create the UI Widgets ---
color_picker = widgets.ColorPicker(
    concise=False,
    description='Pick a color',
    value='white',
    style={'description_width': 'initial'}
)

radius_slider = widgets.IntSlider(
    value=100,
    min=10,
    max=255, # Max possible distance in one channel
    step=5,
    description='Search Radius:',
    style={'description_width': 'initial'}
)

search_button = widgets.Button(
    description='Search',
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    icon='search'
)

# This is where all results will be printed
results_output = widgets.Output()

# --- 4. Define the Search Function (what happens on click) ---
def on_search_button_clicked(b):
    """
    This function runs when the search button is clicked.
    It performs the radius search and displays results.
    """
    # 1. Clear any old results
    results_output.clear_output(wait=True)
    
    with results_output:
        if df.empty:
            print("Database is empty. Nothing to search.")
            return

        # 2. Get values from the widgets
        hex_color = color_picker.value
        radius = radius_slider.value
        target_rgb = np.array(hex_to_rgb(hex_color))

        print(f"Searching for color {target_rgb} within a radius of {radius}...")
        
        # 3. Calculate distance for ALL images at once (very fast)
        # Get all RGB values from the DataFrame
        db_rgbs = df[['R', 'G', 'B']].values 
        
        # Calculate Euclidean distance
        # np.sum((db_rgbs - target_rgb)**2, axis=1) -> calculates (r1-r2)^2 + (g1-g2)^2 + (b1-b2)^2
        # np.sqrt(...) -> takes the square root
        distances = np.sqrt(np.sum((db_rgbs - target_rgb) ** 2, axis=1))
        
        # 4. Filter the DataFrame
        # Select all rows where the distance is within our radius
        results_df = df[distances <= radius].copy()
        
        if results_df.empty:
            print("No matches found.")
            return
            
        # 5. Sort and Display
        # We can add the 'dist' column for sorting
        results_df['dist'] = distances[distances <= radius]
        results_df_sorted = results_df.sort_values(['dist', 'timestamp'], ascending=[True, False])

        print(f"Found {len(results_df_sorted)} matches:")
        
        for _, row in results_df_sorted.iterrows():
            date_str = datetime.fromtimestamp(row['timestamp']).strftime('%Y-%m-%d')
            print(f"\n🖼️ {row['path']} | Dist: {row['dist']:.0f} | Date: {date_str}")
            
            if os.path.exists(row["path"]):
                display(Image(filename=row["path"], width=200))
            else:
                print(f"   (Image file missing: {row['path']})")

# --- 5. Link the button to the function ---
search_button.on_click(on_search_button_clicked)

# --- 6. Display the UI ---
# HBox places widgets horizontally
input_controls = HBox([color_picker, radius_slider, search_button])
# VBox places them vertically
display(VBox([input_controls, results_output]))

Loading database for search...
